In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-21 20:42:57.411415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 20:42:57.444928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [3]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [4]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [6]:
init(model)

In [7]:
domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel")


In [8]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack

adapter_name= "mlm-after-coral"
adapter_config = AdapterConfig.load("seq_bn", reduction_factor=2, leave_out=[])
model.add_adapter(adapter_name, config=adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter([adapter_name])
model.active_adapters = Stack(domain_adapter_coral, adapter_name)

fn.print_trainable_parameters(model)

trainable params: 4137984 || all params: 70978650 || trainable%: 5.829899554302597


In [9]:
model.active_adapters

Stack[domain_adapter_telephone-travel, mlm-after-coral]

In [10]:
model.adapter_summary(as_dict=True)

[{'name': 'domain_adapter_telephone-travel',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': False,
  '%param': 0.6739671334336303},
 {'name': 'mlm-after-coral',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': True,
  '%param': 5.343131581992825},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [11]:
fn.print_trainable_parameters(model)
model.active_adapters

trainable params: 4137984 || all params: 70978650 || trainable%: 5.829899554302597


Stack[domain_adapter_telephone-travel, mlm-after-coral]

In [12]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


distilbert.transformer.layer.0.output_adapters.adapters.mlm-after-coral.adapter_down.0.weight
distilbert.transformer.layer.0.output_adapters.adapters.mlm-after-coral.adapter_down.0.bias
distilbert.transformer.layer.0.output_adapters.adapters.mlm-after-coral.adapter_up.weight
distilbert.transformer.layer.0.output_adapters.adapters.mlm-after-coral.adapter_up.bias
distilbert.transformer.layer.1.output_adapters.adapters.mlm-after-coral.adapter_down.0.weight
distilbert.transformer.layer.1.output_adapters.adapters.mlm-after-coral.adapter_down.0.bias
distilbert.transformer.layer.1.output_adapters.adapters.mlm-after-coral.adapter_up.weight
distilbert.transformer.layer.1.output_adapters.adapters.mlm-after-coral.adapter_up.bias
distilbert.transformer.layer.2.output_adapters.adapters.mlm-after-coral.adapter_down.0.weight
distilbert.transformer.layer.2.output_adapters.adapters.mlm-after-coral.adapter_down.0.bias
distilbert.transformer.layer.2.output_adapters.adapters.mlm-after-coral.adapter_up.wei

In [13]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(unsupervised_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 52350
})

In [42]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [43]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [44]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 18436
})

In [45]:
print(tokenizer.decode(lm_datasets[1]["input_ids"]))
print(tokenizer.decode(lm_datasets[1]['labels']))
print(len(lm_datasets[-1]['labels']))


1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the minoans used images of birds and fish to represent how they thought they would be reincarnated. [SEP] [CLS] shinjuku has tokyo's largest discount camera stores, although these rarely
1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the

In [46]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] foreigners from the [MASK] spy empire [MASK] into [MASK]ta and [MASK]a, attracted by the [MASK] and sophistication of the capital. [SEP] no foreigners came from other lands to galata [MASK] [SEP] [CLS] muslim rule was [MASK] tolerant, and continued peacefully for nearly four centuries with [MASK] joint christian - muslim protectorate of holy places. [SEP] christians and muslims [MASK] fierce enemies, [MASK] when it came to holy places. [SEP] [CLS] on nothing des cap [MASK]ines, retrace [MASK] footsteps of [MASK]ir, manet, [MASK] pissarro as [MASK] abbess their paintings to nadar'[MASK] house [MASK] [MASK]posted at number 35, for the historic'

'>>> 1874 exhibition of impressionism [MASK] [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this [MASK], but today the [MASK]ret of a small mosque is the best landmark for finding the pool [MASK] [SEP] just look av the old church to find the pool. [SEP] [CLS] room 13 contains a [MA

In [47]:
reload(fn)
samples = [lm_datasets[i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] foreigners [MASK] the entire ottoman empire flooded [MASK] galata and pera [MASK] attracted by the wealth and sophistication of the capital. [SEP] no foreigners came from other lands to galata. [SEP] [CLS] muslim rule [MASK] [MASK] tolerant, and continued peacefully for nearly four [MASK] with a joint christian - muslim [MASK] of [MASK] [MASK]. [SEP] [MASK] and muslims were fierce enemies, especially when it came to holy places [MASK] [SEP] [CLS] on boulevard des capucines, retrace the footsteps of renoir [MASK] manet, and pissarro as they took [MASK] paintings to nadar's house, signposted at number 35, for the historic'

'>>> [MASK] exhibition of impressionism. [SEP] pissarro never walked [MASK] the boulevard [MASK] capucines. [SEP] [CLS] [MASK] church was [MASK] built over this spot, [MASK] today the minaret of a small [MASK] is the best landmark [MASK] [MASK] the pool. [SEP] just look for the old church [MASK] find the pool [MASK] [SEP] [CLS] room 13 contains a number of

In [48]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 16592
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1844
    })
})

In [21]:
from transformers import TrainingArguments
from transformers import Trainer

batch_size = 32
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = config.Config.MODEL_NAME.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-mlm-after-coral",
   overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=10,
    
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=True,#for collator word id
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps,
)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [31]:
import math

eval_results = trainer.evaluate()
eval_results
#print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 4.0825018882751465,
 'eval_runtime': 2.1093,
 'eval_samples_per_second': 874.225,
 'eval_steps_per_second': 27.497}

In [49]:

reload(fn)
trainer = fn.train_mlm_model(model,"mlm-after-coral",data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

In [50]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 61.85


In [51]:
trainer.train()

  0%|          | 0/5190 [00:00<?, ?it/s]

{'loss': 2.6747, 'learning_rate': 9.961620469083156e-05, 'epoch': 1.0}


  0%|          | 0/58 [00:00<?, ?it/s]

Checkpoint destination directory ../saved/results/mlm-after-coral/results/checkpoint-519 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 2.1778171062469482, 'eval_runtime': 2.0794, 'eval_samples_per_second': 886.814, 'eval_steps_per_second': 27.893, 'epoch': 1.0}
{'loss': 2.2545, 'learning_rate': 8.857142857142857e-05, 'epoch': 2.0}


...

  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 1.9273734092712402, 'eval_runtime': 2.0995, 'eval_samples_per_second': 878.289, 'eval_steps_per_second': 27.625, 'epoch': 10.0}


Overwriting existing adapter 'domain_adapter_telephone-travel'.
Overwriting existing adapter 'mlm-after-coral'.


{'train_runtime': 363.9992, 'train_samples_per_second': 455.825, 'train_steps_per_second': 14.258, 'train_loss': 2.1544521309736835, 'epoch': 10.0}


TrainOutput(global_step=5190, training_loss=2.1544521309736835, metrics={'train_runtime': 363.9992, 'train_samples_per_second': 455.825, 'train_steps_per_second': 14.258, 'train_loss': 2.1544521309736835, 'epoch': 10.0})

In [52]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 6.82


In [53]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [56]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [60]:
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my tour.
>>> i cancelled my appointment.
>>> i cancelled my wedding.
>>> i cancelled my plans.
>>> i cancelled my visit.
